In [42]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import sentence_bleu

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:

df = pd.read_csv("/content/drive/MyDrive/test.csv")
df.head()
df.shape
df_train = df[:500]
df_train.shape
df_test = df[500:600]
df_train.head()
df_test.shape
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps = PorterStemmer()
def clean_text(review):
    review = re.sub('[^a-zA-Z0-1]', ' ', review)
    review = review.lower()
    review = review.split()


    review = ' '.join(review)
    return review

In [44]:
article_train = df_train['article'].apply(clean_text).values.tolist()
highlights_train = df_train['highlights'].apply(clean_text).values.tolist()
article_test = df_test['article'].apply(clean_text).values.tolist()
highlights_test = df_test['highlights'].apply(clean_text).values.tolist()

In [45]:
#article_train = df_train['article'].values.tolist()
#highlights_train = df_train['highlights'].values.tolist()
#article_test = df_test['article'].values.tolist()
#highlights_test = df_test['highlights'].values.tolist()

In [46]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [47]:
nltk.download('stopwords')

# Assuming you have already processed your data and created a list of articles
# For simplicity, let's assume article_train and article_test are already prepared

# Tokenize articles into sentences
sentences_train = [word_tokenize(sent.lower()) for article in article_train for sent in sent_tokenize(article)]
sentences_test = [word_tokenize(sent.lower()) for article in article_test for sent in sent_tokenize(article)]

# Flatten the list of sentences for TF-IDF vectorization
flat_sentences_train = [' '.join(sent) for sent in sentences_train]
flat_sentences_test = [' '.join(sent) for sent in sentences_test]

# Create TF-IDF vectors after removing stop words
stop_words = set(stopwords.words('english'))

vectorizer = TfidfVectorizer(stop_words='english')  # Set stop_words to 'english'
tfidf_matrix_train = vectorizer.fit_transform(flat_sentences_train)
tfidf_matrix_test = vectorizer.transform(flat_sentences_test)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
# Function to generate summaries using TF-IDF
def generate_summary(article, num_words=50):
    article_tfidf = vectorizer.transform([article.lower()])

    # Get the indices of the most important sentences based on TF-IDF values
    important_sentence_indices = article_tfidf.indices

    # Filter out indices that are out of range
    important_sentence_indices = [i for i in important_sentence_indices if i < len(flat_sentences_train)]

    # Sort the remaining indices
    important_sentence_indices = sorted(important_sentence_indices)

    # Extract the important sentences from the original article
    important_sentences = [flat_sentences_train[i] for i in important_sentence_indices]

    # Adjust the number of words based on the available sentences
    num_words = min(num_words, len(important_sentence_indices))

    # Combine the important sentences to form the summary
    summary = ' '.join(important_sentences[:num_words])

    return summary

# Generate summaries for the test set
predicted_summaries = [generate_summary(article) for article in article_test]

In [49]:

# Calculate cosine similarity
cosine_similarities = []
for actual, predicted in zip(highlights_test, predicted_summaries):
    actual_tfidf = vectorizer.transform([actual.lower()])
    predicted_tfidf = vectorizer.transform([predicted.lower()])

    # Calculate cosine similarity
    cosine_sim = cosine_similarity(actual_tfidf, predicted_tfidf)[0][0]
    cosine_similarities.append(cosine_sim)

# Mean cosine similarity
mean_cosine_similarity = sum(cosine_similarities) / len(cosine_similarities)
print("Mean Cosine Similarity:", mean_cosine_similarity)


Mean Cosine Similarity: 0.021815581391597116
